In [10]:
from collections import defaultdict
from datetime import datetime
import torch
import numpy as np
import pandas as pd
import datetime as dt

In [11]:
m1_1m_path = '../../../datasets/ml-1m'
PATH_TO_PROCESSED_DATA = 'preprocessed_data/'

In [12]:
file_path = m1_1m_path + '/ratings.dat'

data_ratings = pd.read_csv(file_path, sep='::', engine='python',
                   names=['SessionID', 'ItemID', 'Rating', 'Time'], index_col='SessionID').drop('Rating', axis=1, inplace=False)

In [13]:
data_ratings.loc[2]

,ItemID,Time
SessionID,,
2,1357,978298709
2,3068,978299000
2,1537,978299620
2,647,978299351
2,2194,978299297
...,...,...
2,356,978299686
2,1245,978299200
2,1246,978299418


## GRU4Rec handling
Only use implicit feedback in the sequence of items

In [14]:
order_ratings = data_ratings
# session_lengths>1
order_ratings = order_ratings.groupby('SessionID').filter(lambda  x: len(x) > 1)
order_ratings = order_ratings.groupby('ItemID').filter(lambda  x: len(x) >= 5)
order_ratings = order_ratings.groupby('SessionID').filter(lambda  x: len(x) >= 2)
order_ratings = order_ratings.groupby('SessionID').apply(lambda  x: x.sort_values('Time'))
# We discard users and items with fewer than 5 related actions.
order_ratings

C:\Users\Yunlong\AppData\Local\Temp\ipykernel_15864\3072087427.py:6: FutureWarning: Not prepending group keys to the result index of transform-like apply. In the future, the group keys will be included in the index, regardless of whether the applied function returns a like-indexed object.
To preserve the previous behavior, use

	>>> .groupby(..., group_keys=False)

To adopt the future behavior and silence this warning, use 

	>>> .groupby(..., group_keys=True)
  order_ratings = order_ratings.groupby('SessionID').apply(lambda  x: x.sort_values('Time'))


,ItemID,Time
SessionID,,
1,3186,978300019
1,1721,978300055
1,1022,978300055
1,1270,978300055
1,2340,978300103
...,...,...
6040,2917,997454429
6040,1784,997454464
6040,1921,997454464


In [15]:
len(set(order_ratings.index))

6040

In [16]:
tmax = order_ratings.Time.max()
interval = 86400 * 30 * 12 * 2
split_point_1 = tmax - interval
session_train = order_ratings[order_ratings['Time'] <= split_point_1]
session_test = order_ratings[order_ratings['Time'] > split_point_1]
session_test = session_test.groupby('SessionID').filter(lambda  x: len(x) >= 2)
len(set(session_train.index)), len(set(session_test.index)), len(session_train), len(session_test)

(6035, 914, 932987, 66579)

In [17]:
# train = session_train.reset_index().rename(columns={'index': 'SessionID'})
# train

In [18]:
train = session_train
test = session_test
print('Full train set\n\tEvents: {}\n\tSessions: {}\n\tItems: {}'.format(len(train), train.index.nunique(), train.ItemID.nunique()))
train.to_csv(PATH_TO_PROCESSED_DATA + 'ml-1m_train_full.txt', sep=',', index=True)
print('Test set\n\tEvents: {}\n\tSessions: {}\n\tItems: {}'.format(len(test), test.index.nunique(), test.ItemID.nunique()))
test.to_csv(PATH_TO_PROCESSED_DATA + 'ml-1m_test.txt', sep=',', index=True)

Full train set
	Events: 932987
	Sessions: 6035
	Items: 3414
Test set
	Events: 66579
	Sessions: 914
	Items: 3246


In [19]:
tmax = train.Time.max()
interval = 86400 * 30 * 3
split_point_2 = tmax - interval
session_train = train[train['Time'] <= split_point_2]
session_val = train[train['Time'] > split_point_2]
train_tr = session_train
valid = session_val
valid = valid.groupby('SessionID').filter(lambda  x: len(x) >= 2)
print('Train set\n\tEvents: {}\n\tSessions: {}\n\tItems: {}'.format(len(train_tr), train_tr.index.nunique(), train_tr.ItemID.nunique()))
train_tr.to_csv(PATH_TO_PROCESSED_DATA + 'ml-1m_train_tr.txt', sep=',', index=True)
print('Validation set\n\tEvents: {}\n\tSessions: {}\n\tItems: {}'.format(len(valid), valid.index.nunique(), valid.ItemID.nunique()))
valid.to_csv(PATH_TO_PROCESSED_DATA + 'ml-1m_train_valid.txt', sep=',', index=True)

Train set
	Events: 850937
	Sessions: 5719
	Items: 3411
Validation set
	Events: 81984
	Sessions: 1100
	Items: 3239
